# ⭐ Contrucción del Modelo Estrella 

## 1. Carga de los DataFrame ya Transformados y Limpios

In [5]:
import pandas as pd
import numpy as np
import os 

PROCESSED_DATA_PATH = 'data/processed/tecnoStore'

def load_parquet_file(filename):
    """Carga un archivo Parquet desde la ruta predefinida."""
    full_path = os.path.join(PROCESSED_DATA_PATH, filename)
    try:
        df = pd.read_parquet(full_path)
        print(f"✔️ Cargado: {filename}")
        return df
    except FileNotFoundError:
        print(f"❌ Error: Archivo no encontrado en la ruta {full_path}. Asegúrate de que los archivos Parquet existen.")
        return None

# Cargar las tablas ETL ya limpias y transformadas
df_ventas = load_parquet_file('fact_ventas.parquet')
df_fecha = load_parquet_file('dim_calendario.parquet')
df_productos = load_parquet_file('dim_productos.parquet')
df_clientes = load_parquet_file('dim_clientes.parquet')
df_sucursales = load_parquet_file('dim_sucursales.parquet') 
df_canales = load_parquet_file('dim_canales.parquet')

# Verificar si todas las tablas se cargaron correctamente
if any(df is None for df in [df_ventas, df_fecha, df_productos, df_clientes, df_sucursales, df_canales]):
    print("\nDeteniendo el proceso. Corrige las rutas o nombres de archivo Parquet.")
    exit()

✔️ Cargado: fact_ventas.parquet
✔️ Cargado: dim_calendario.parquet
✔️ Cargado: dim_productos.parquet
✔️ Cargado: dim_clientes.parquet
✔️ Cargado: dim_sucursales.parquet
✔️ Cargado: dim_canales.parquet


## 2. Conversión de Tipos (Asegurar Claves de Unión)

In [10]:
# Es buena práctica asegurar que las claves de unión tengan el tipo de dato correcto
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'])
df_fecha['fecha'] = pd.to_datetime(df_fecha['fecha'])

# Asegurar que las claves ID sean tipo entero para uniones eficientes
id_cols = ['producto_id', 'cliente_id', 'canal_id']
for col in id_cols:
    df_ventas[col] = df_ventas[col].astype('Int64')
    if col in df_productos.columns: df_productos[col] = df_productos[col].astype('Int64')
    if col in df_clientes.columns: df_clientes[col] = df_clientes[col].astype('Int64')
    if col in df_canales.columns: df_canales[col] = df_canales[col].astype('Int64')

# Muestra los tipos de datos de las columnas clave en la tabla de hechos
print("--- Tipos de Datos en df_ventas ---")
print(df_ventas[['fecha', 'producto_id', 'cliente_id', 'canal_id']].dtypes)

# Muestra los tipos de datos en las dimensiones
print("\n--- Tipos de Datos en df_calendario (Clave) ---")
print(df_fecha[['fecha']].dtypes)

print("\n--- Tipos de Datos en df_productos (Clave) ---")
print(df_productos[['producto_id']].dtypes)

print("\n--- Tipos de Datos en df_clientes (Clave) ---")
print(df_clientes[['cliente_id']].dtypes)

print("\n--- Tipos de Datos en df_canales (Clave) ---")
print(df_canales[['canal_id']].dtypes)

--- Tipos de Datos en df_ventas ---
fecha          datetime64[ns]
producto_id             Int64
cliente_id              Int64
canal_id                Int64
dtype: object

--- Tipos de Datos en df_calendario (Clave) ---
fecha    datetime64[ns]
dtype: object

--- Tipos de Datos en df_productos (Clave) ---
producto_id    Int64
dtype: object

--- Tipos de Datos en df_clientes (Clave) ---
cliente_id    Int64
dtype: object

--- Tipos de Datos en df_canales (Clave) ---
canal_id    Int64
dtype: object


In [11]:

print("\n--- Inspección Detallada de df_ventas (.info()) ---")
df_ventas.info()

print("\n--- Inspección Detallada de df_productos (.info()) ---")
df_productos.info()


--- Inspección Detallada de df_ventas (.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224678 entries, 0 to 224677
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   transaction_id   224678 non-null  int64         
 1   fecha            224678 non-null  datetime64[ns]
 2   cliente_id       224678 non-null  Int64         
 3   producto_id      224678 non-null  Int64         
 4   canal_id         224678 non-null  Int64         
 5   sucursal_id      224678 non-null  float64       
 6   cantidad         224678 non-null  int64         
 7   monto_venta_usd  224678 non-null  float64       
dtypes: Int64(3), datetime64[ns](1), float64(2), int64(2)
memory usage: 14.4 MB

--- Inspección Detallada de df_productos (.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  -----

In [12]:
def check_key_consistency(df_left, key_left, df_right, key_right):
    """Verifica si los tipos de datos de las claves de unión coinciden."""
    type_left = df_left[key_left].dtype
    type_right = df_right[key_right].dtype
    
    if type_left == type_right:
        print(f"✔️ Éxito: Las claves '{key_left}' y '{key_right}' coinciden. Tipo: {type_left}")
    else:
        print(f"❌ Advertencia: Tipos de claves no coinciden. {key_left}: {type_left} vs. {key_right}: {type_right}")

In [39]:

# Ejemplo de uso:
check_key_consistency(df_ventas, 'fecha', df_fecha, 'fecha')
check_key_consistency(df_ventas, 'producto_id', df_productos, 'producto_id')
check_key_consistency(df_ventas, 'cliente_id', df_clientes, 'cliente_id')
check_key_consistency(df_ventas, 'canal_id', df_canales, 'canal_id')
check_key_consistency(df_ventas, 'sucursal_id', df_sucursales, 'sucursal_id')


✔️ Éxito: Las claves 'fecha' y 'fecha' coinciden. Tipo: datetime64[ns]
✔️ Éxito: Las claves 'producto_id' y 'producto_id' coinciden. Tipo: Int64
✔️ Éxito: Las claves 'cliente_id' y 'cliente_id' coinciden. Tipo: Int64
✔️ Éxito: Las claves 'canal_id' y 'canal_id' coinciden. Tipo: Int64
❌ Advertencia: Tipos de claves no coinciden. sucursal_id: float64 vs. sucursal_id: int64


## 3. Construcción del Modelo Estrella (Uniones/Joins)

In [14]:
print("\n⭐ Iniciando la construcción del Modelo Estrella...")

# Partimos de la tabla de Hechos (df_ventas)
df_modelo_estrella = df_ventas.copy()


⭐ Iniciando la construcción del Modelo Estrella...


### 3.1. Unir a Dimensión Fecha

In [17]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_fecha.drop(columns=['año', 'mes', 'día'], errors='ignore'), # Evitar duplicar columnas si ya se hicieron las uniones antes
    on='fecha',
    how='left'
)
print("   -> Unido con dim_fecha.")

   -> Unido con dim_fecha.


In [ ]:
df_modelo_estrella.head(3)

,transaction_id,fecha,cliente_id,producto_id,canal_id,sucursal_id,cantidad,monto_venta_usd,nombre_mes,trimestre,semana_año,nombre_día
0,1,2018-01-01,2012,5,2,17.0,1,219.02,Enero,T1,1,Lunes
1,2,2018-01-01,3415,15,2,7.0,1,77.72,Enero,T1,1,Lunes
2,3,2018-01-01,1093,17,2,15.0,1,100.28,Enero,T1,1,Lunes
3,4,2018-01-01,2567,10,2,9.0,1,1051.77,Enero,T1,1,Lunes
4,5,2018-01-01,3274,11,2,14.0,1,800.92,Enero,T1,1,Lunes


### 3.2. Unir a Dimensión Productos

In [20]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_productos,
    on='producto_id',
    how='left'
)
print("   -> Unido con dim_productos.")

   -> Unido con dim_productos.


In [22]:
df_modelo_estrella.head(3)

,transaction_id,fecha,cliente_id,producto_id,canal_id,sucursal_id,cantidad,monto_venta_usd,nombre_mes,trimestre,semana_año,nombre_día,nombre_producto,categoria_producto,precio_unitario_usd
0,1,2018-01-01,2012,5,2,17.0,1,219.02,Enero,T1,1,Lunes,Impresora Multifunción,Tecnologia_Remoto,220
1,2,2018-01-01,3415,15,2,7.0,1,77.72,Enero,T1,1,Lunes,Parlante Bluetooth,Accesorios_Entretenimiento,80
2,3,2018-01-01,1093,17,2,15.0,1,100.28,Enero,T1,1,Lunes,Cafetera Espresso,Linea_Blanca,100


### 3.3. Unir a Dimensión Clientes

In [27]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_clientes,
    on='cliente_id',
    how='left'
)
print("   -> Unido con dim_clientes.")

   -> Unido con dim_clientes.


In [28]:
df_modelo_estrella.head(3)

,transaction_id,fecha,cliente_id,producto_id,canal_id,sucursal_id,cantidad,monto_venta_usd,nombre_mes,trimestre,...,categoria_producto,precio_unitario_usd,nombre_cliente_x,email_cliente_x,ciudad_cliente_x,segmento_cliente_x,nombre_cliente_y,email_cliente_y,ciudad_cliente_y,segmento_cliente_y
0,1,2018-01-01,2012,5,2,17.0,1,219.02,Enero,T1,...,Tecnologia_Remoto,220,Alma Ortiz Rodriguez,tgimenez@example.net,Río Gallegos,Frecuente,Alma Ortiz Rodriguez,tgimenez@example.net,Río Gallegos,Frecuente
1,2,2018-01-01,3415,15,2,7.0,1,77.72,Enero,T1,...,Accesorios_Entretenimiento,80,Sr(a). Antonella Cordoba,suarezjulia@example.org,Mar del Plata,Ocasional,Sr(a). Antonella Cordoba,suarezjulia@example.org,Mar del Plata,Ocasional
2,3,2018-01-01,1093,17,2,15.0,1,100.28,Enero,T1,...,Linea_Blanca,100,Camilo Alma Morales,juan-cruz47@example.org,Neuquén,Frecuente,Camilo Alma Morales,juan-cruz47@example.org,Neuquén,Frecuente


### 3.4. Unir a Dimensión Canales

In [29]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_canales,
    on='canal_id',
    how='left'
)
print("   -> Unido con dim_canales.")

   -> Unido con dim_canales.


In [30]:
df_modelo_estrella.head(3)

,transaction_id,fecha,cliente_id,producto_id,canal_id,sucursal_id,cantidad,monto_venta_usd,nombre_mes,trimestre,...,precio_unitario_usd,nombre_cliente_x,email_cliente_x,ciudad_cliente_x,segmento_cliente_x,nombre_cliente_y,email_cliente_y,ciudad_cliente_y,segmento_cliente_y,nombre_canal
0,1,2018-01-01,2012,5,2,17.0,1,219.02,Enero,T1,...,220,Alma Ortiz Rodriguez,tgimenez@example.net,Río Gallegos,Frecuente,Alma Ortiz Rodriguez,tgimenez@example.net,Río Gallegos,Frecuente,Sucursal Fisica
1,2,2018-01-01,3415,15,2,7.0,1,77.72,Enero,T1,...,80,Sr(a). Antonella Cordoba,suarezjulia@example.org,Mar del Plata,Ocasional,Sr(a). Antonella Cordoba,suarezjulia@example.org,Mar del Plata,Ocasional,Sucursal Fisica
2,3,2018-01-01,1093,17,2,15.0,1,100.28,Enero,T1,...,100,Camilo Alma Morales,juan-cruz47@example.org,Neuquén,Frecuente,Camilo Alma Morales,juan-cruz47@example.org,Neuquén,Frecuente,Sucursal Fisica


### 3.5. Unir a Dimensión Sucursales

In [33]:
# Agregar fila especial para ventas online en la dimensión sucursales
df_sucursales_online = pd.DataFrame([{
    "sucursal_id": 0,
    "provincia_sucursal": "Venta Online",
    "nombre_sucursal": "N/A Online"
}])

# Concatenar la fila especial a la dimensión sucursales
df_sucursales = pd.concat([df_sucursales, df_sucursales_online], ignore_index=True)

# Left join para mantener todas las ventas
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_sucursales,
    on="sucursal_id",
    how="left"
)

print("   -> Unido con dim_sucursales incluyendo el canal Online.")


   -> Unido con dim_sucursales incluyendo el canal Online.


In [34]:
df_modelo_estrella.head(3)

,transaction_id,fecha,cliente_id,producto_id,canal_id,sucursal_id,cantidad,monto_venta_usd,nombre_mes,trimestre,...,email_cliente_y,ciudad_cliente_y,segmento_cliente_y,nombre_canal,nombre_sucursal_x,direccion_sucursal_x,provincia_sucursal_x,nombre_sucursal_y,direccion_sucursal_y,provincia_sucursal_y
0,1,2018-01-01,2012,5,2,17.0,1,219.02,Enero,T1,...,tgimenez@example.net,Río Gallegos,Frecuente,Sucursal Fisica,Sucursal Q,"Calle Buenos Aires N° 12 Torre 1 Dto. 7, La Pl...",Corrientes,Sucursal Q,"Calle Buenos Aires N° 12 Torre 1 Dto. 7, La Pl...",Corrientes
1,2,2018-01-01,3415,15,2,7.0,1,77.72,Enero,T1,...,suarezjulia@example.org,Mar del Plata,Ocasional,Sucursal Fisica,Sucursal G,"Avenida 1 N° 79 Dto. 8, Rosario 2000, Santa Fe",Neuquén,Sucursal G,"Avenida 1 N° 79 Dto. 8, Rosario 2000, Santa Fe",Neuquén
2,3,2018-01-01,1093,17,2,15.0,1,100.28,Enero,T1,...,juan-cruz47@example.org,Neuquén,Frecuente,Sucursal Fisica,Sucursal O,"Av. Santa Fe N° 959 Oficina 1, La Rioja 5300, ...",Tierra del Fuego,Sucursal O,"Av. Santa Fe N° 959 Oficina 1, La Rioja 5300, ...",Tierra del Fuego


## 4. Selección y Limpieza Final

In [36]:
# Renombrar columnas clave para la historia de datos
df_modelo_estrella.rename(columns={
    'monto_venta_usd': 'Monto_Venta_USD',
    'nombre_canal': 'Canal_Venta',
    'provincia_sucursal': 'Provincia'
}, inplace=True)

# Eliminar IDs intermedios si ya no son necesarios (o mantenerlos para drill-down)
# df_modelo_estrella = df_modelo_estrella.drop(columns=['cliente_id', 'producto_id', 'canal_id', 'sucursal_id', 'fecha', 'dia', 'mes', 'año'], errors='ignore')

# Mostrar las primeras filas y columnas clave para validar
print("\n✔️ ¡Modelo Estrella consolidado finalizado!")
print(f"Número total de transacciones: {len(df_modelo_estrella):,}")
print("Columnas clave del DataFrame final:")
print(df_modelo_estrella.head(3).T)


✔️ ¡Modelo Estrella consolidado finalizado!
Número total de transacciones: 224,678
Columnas clave del DataFrame final:
                                                                      0  \
transaction_id                                                        1   
fecha                                               2018-01-01 00:00:00   
cliente_id                                                         2012   
producto_id                                                           5   
canal_id                                                              2   
sucursal_id                                                        17.0   
cantidad                                                              1   
Monto_Venta_USD                                                  219.02   
nombre_mes                                                        Enero   
trimestre                                                            T1   
semana_año                                             

## 5. Exportar el DataFrame Consolidado

In [37]:
# Puedes exportar el modelo estrella final de nuevo a Parquet o a un formato más simple
FINAL_OUTPUT_PATH = 'data/analytical_layer/tecnoStore/modelo_estrella_final.parquet'
os.makedirs(os.path.dirname(FINAL_OUTPUT_PATH), exist_ok=True)
df_modelo_estrella.to_parquet(FINAL_OUTPUT_PATH, index=False)
print(f"\n💾 DataFrame final exportado a: {FINAL_OUTPUT_PATH}")


💾 DataFrame final exportado a: data/analytical_layer/tecnoStore/modelo_estrella_final.parquet
